In [7]:
!pip install emoji

In [8]:
!pip install nltk

In [9]:
!pip install contractions

In [10]:
import pandas as pd
import re
import emoji

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.util import mark_negation
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
headers = ["id", "company", "sentiment", "tweet"]
df_train = pd.read_csv('/content/twitter_training.csv', names=headers)
df_val = pd.read_csv('/content/twitter_validation.csv', names=headers)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [13]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [14]:
# Step 1: Remove NaN values
def remove_nan_values(df, columns):
    for column in columns:
        df = df[df[column].notna()]  # Remove rows with NaN values
        df = df[df[column].astype(bool)]  # Remove rows with empty lists or lists with only white spaces
    return df

# Step 2: Assign tweet labels
def one_hot_encode_sentiments(sentiment): #make neutral and irrelavent same number, and look over
    if sentiment == 'Positive':
        return 2
    elif sentiment == 'Negative':
        return 0
    elif sentiment == 'Neutral':
        return 1
    elif sentiment == 'Irrelevant':
        return 1
    else:
      return '?><?'

def expand_slang(text):
    tokens = word_tokenize(text)
    tokens = [abbreviations.get(word.lower(), word) for word in tokens]
    text = ' '.join(tokens)
    return text


# Step 3: Clean tweets
def clean_tweets(tweet):
    # Remove @mentions
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)

    # Remove hashtags
    tweet = re.sub(r'#\w+', '', tweet)

    # Convert emojis to text
    tweet = emoji.demojize(tweet)

    # Expand contractions
    tweet = contractions.fix(tweet)
    ##############################3

    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)

    # Remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Remove excessive whitespace
    tweet = re.sub(r'\s+', ' ', tweet)

    # Standardize repeated characters
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)

    #Expand slang terms
    tweet = expand_slang(tweet)

    # Tokenize the tweet
    tokens = word_tokenize(tweet)

    # Lowercase and normalize
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Handle abbreviations and acronyms
    abbreviation_words = {"lol": "laughing out loud", "btw": "by the way"}  # Add more as needed
    tokens = [abbreviation_words.get(token.lower(), token) for token in tokens]

    # Handle negation using mark_negation
    #negation_tokens = mark_negation(tokens)
    #tweet = "I don't like this product. It is not good."
    #['I', "don't", 'like_NEG', 'this_NEG', 'product_NEG', '.', 'It_NEG', 'is_NEG', 'not_NEG', 'good_NEG', '.']

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Uncomment the following lines to include POS tags as comments
    #pos_tags = nltk.pos_tag(tokens)
    #tokens_with_pos = [f"{token}/{pos}" for token, pos in pos_tags]

    return tokens

In [15]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [16]:
# Step 2: Assign tweet labels and remove rows with 'Irrelevant' sentiment
df_train['sentiment'] = df_train['sentiment'].apply(one_hot_encode_sentiments)
df_val['sentiment'] = df_val['sentiment'].apply(one_hot_encode_sentiments)

# Remove rows with 'Irrelevant' sentiment from the DataFrame
df_train = df_train.dropna(subset=['sentiment'])
df_val = df_val.dropna(subset=['sentiment'])

print(df_train.head())
print(df_val.head())

     id      company  sentiment  \
0  2401  Borderlands          2   
1  2401  Borderlands          2   
2  2401  Borderlands          2   
3  2401  Borderlands          2   
4  2401  Borderlands          2   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company  sentiment  \
0  3364   Facebook          1   
1   352     Amazon          1   
2  8312  Microsoft          0   
3  4371      CS-GO          0   
4  4433     Google          1   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [17]:
# Step 3: Clean tweets
df_train['tweet'] = df_train['tweet'].apply(clean_tweets)
df_val['tweet'] = df_val['tweet'].apply(clean_tweets)

print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          2  [getting, borderland, murder]
1  2401  Borderlands          2         [coming, border, kill]
2  2401  Borderlands          2    [getting, borderland, kill]
3  2401  Borderlands          2   [coming, borderland, murder]
4  2401  Borderlands          2  [getting, borderland, murder]
     id    company  sentiment  \
0  3364   Facebook          1   
1   352     Amazon          1   
2  8312  Microsoft          0   
3  4371      CS-GO          0   
4  4433     Google          1   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full, closet, hacking, tru...  
4  [president, slapping, american, face, really, ...  


In [18]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          2  [getting, borderland, murder]
1  2401  Borderlands          2         [coming, border, kill]
2  2401  Borderlands          2    [getting, borderland, kill]
3  2401  Borderlands          2   [coming, borderland, murder]
4  2401  Borderlands          2  [getting, borderland, murder]
      id               company  sentiment  \
0   3364              Facebook          1   
1    352                Amazon          1   
4   4433                Google          1   
6   7925             MaddenNFL          2   
7  11332  TomClancysRainbowSix          2   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
4  [president, slapping, american, face, really, ...  
6  [thank, new, te, austin, hooper, orange, brown...  
7  [rocket, league, sea, thief, rainbow, six, sie...  


In [19]:
# Print the resulting DataFrame
print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          2  [getting, borderland, murder]
1  2401  Borderlands          2         [coming, border, kill]
2  2401  Borderlands          2    [getting, borderland, kill]
3  2401  Borderlands          2   [coming, borderland, murder]
4  2401  Borderlands          2  [getting, borderland, murder]
      id               company  sentiment  \
0   3364              Facebook          1   
1    352                Amazon          1   
4   4433                Google          1   
6   7925             MaddenNFL          2   
7  11332  TomClancysRainbowSix          2   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
4  [president, slapping, american, face, really, ...  
6  [thank, new, te, austin, hooper, orange, brown...  
7  [rocket, league, sea, thief, rainbow, six, sie...  


In [20]:
# Assuming your DataFrame is named df
import torch
# Drop the 'id' column
df_train_new = df_train.drop('id', axis=1)
df_val_new = df_val.drop('id', axis=1)

# Export the DataFrame to a CSV file
df_train_new.to_csv('df_train_new_int.csv', index=False)
df_val_new.to_csv('df_val_new_int.csv', index=False)

In [21]:
print(df_train_new[:5])

       company  sentiment                          tweet
0  Borderlands          2  [getting, borderland, murder]
1  Borderlands          2         [coming, border, kill]
2  Borderlands          2    [getting, borderland, kill]
3  Borderlands          2   [coming, borderland, murder]
4  Borderlands          2  [getting, borderland, murder]


In [22]:
import torch
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
# Assuming you have a series of lists in your DataFrame column 'tweet'
# For example, assuming your DataFrame is named 'df_train_new' and 'df_val_new', and the column with lists is 'tweet'

#### Where we can add nick pre process###
series_of_lists_train = df_train_new['tweet']
series_of_lists_valid = df_val_new['tweet']

# Step 1: Flatten the series of lists into a single list for both training and validation sets
all_words = [word for sublist in series_of_lists_train for word in sublist]
all_words_valid = [word for sublist in series_of_lists_valid for word in sublist]

# Combine the words from both training and validation sets
all_words.extend(all_words_valid)

# Step 2: Count the occurrences of each word
word_counts = Counter(all_words)

# Step 3: Sort the words based on their frequency in descending order
sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Step 4: Create a vocabulary dictionary with words ordered by frequency
vocab_dict = {word: idx for idx, (word, _) in enumerate(sorted_words)}

# Step 5: Convert the series of lists into sequences of indices using the vocabulary dictionary
sequences_of_indices_train = [[vocab_dict[word] for word in sublist] for sublist in series_of_lists_train]
sequences_of_indices_valid = [[vocab_dict[word] for word in sublist] for sublist in series_of_lists_valid]

# Step 6: Pad the sequences with zeros to make them all the same length
# Find the maximum length of a sequence in both training and validation sets combined
max_seq_length_train = max(len(seq) for seq in sequences_of_indices_train)
max_seq_length_valid = max(len(seq) for seq in sequences_of_indices_valid)
max_seq_length = max(max_seq_length_train, max_seq_length_valid)

padded_sequences_train = pad_sequence([torch.tensor(seq[:max_seq_length]) for seq in sequences_of_indices_train], batch_first=True, padding_value=0)
padded_sequences_valid = pad_sequence([torch.tensor(seq + [0] * (max_seq_length - len(seq))) for seq in sequences_of_indices_valid], batch_first=True, padding_value=0)


# Ensure the shapes of both padded sequences
print(padded_sequences_train.shape)
print(padded_sequences_valid.shape)


torch.Size([50380, 198])
torch.Size([734, 198])


In [23]:
# train_labels_list = torch.tensor(df_train_new['sentiment'])
# val_labels_list = torch.tensor(df_val_new['sentiment'])
train_label_list = []
valid_label_list = []
for label in df_train_new['sentiment']:
  train_label_list.append(label)
for label in df_val_new['sentiment']:
  valid_label_list.append(label)
train_labels_tensor = torch.tensor(train_label_list)
valid_labels_tensor = torch.tensor(valid_label_list)

In [24]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, labels, h0_c0=None):
        self.sequences = sequences
        self.labels = labels
        self.h0_c0 = h0_c0

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]

        if self.h0_c0 is not None:
            h0_c0 = self.h0_c0[idx]
            return sequence, label, h0_c0
        else:
            return sequence, label, None

def collate_fn(batch):
    # Separate the elements of the batch into individual lists
    sequences, labels, h0_c0 = zip(*batch)

    # Pad the sequences with zeros to make them the same length
    padded_sequences = pad_sequence(sequences, batch_first=True)

    # Stack the tensors
    labels = torch.tensor(labels)

    # Filter out None values in h0_c0
    h0_c0 = [tensor for tensor in h0_c0 if tensor is not None]

    if h0_c0:
        # Separate h0 and c0 tensors
        h0 = torch.stack([tensor[0] for tensor in h0_c0])
        c0 = torch.stack([tensor[1] for tensor in h0_c0])
    else:
        # If h0_c0 is empty, create tensors of zeros with appropriate shapes
        batch_size = padded_sequences.size(0)
        num_layers = 1  # Set the appropriate num_layers
        hidden_size = 128  # Set the appropriate hidden_size

        h0 = torch.zeros(num_layers, batch_size, hidden_size).to(padded_sequences.device)
        c0 = torch.zeros(num_layers, batch_size, hidden_size).to(padded_sequences.device)

    return padded_sequences, labels, h0, c0
# ... (your data preprocessing steps) ...

# Convert sequences and labels to torch.Tensor
sequences_tensor = [torch.tensor(seq) for seq in sequences_of_indices_train]
labels_tensor = torch.tensor(train_label_list)

# Create custom dataset and data loader with collate_fn
train_dataset = CustomDataset(sequences_tensor, labels_tensor)
batch_size = 32
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [25]:
prepared_train_dataset = torch.utils.data.TensorDataset(padded_sequences_train, train_labels_tensor)
prepared_valid_dataset = torch.utils.data.TensorDataset(padded_sequences_valid, valid_labels_tensor)

train_data_loader = torch.utils.data.DataLoader(prepared_train_dataset, batch_size=32, shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(prepared_valid_dataset, batch_size=32, shuffle=True)

In [26]:
for batch_data, batch_labels in valid_data_loader:
    print("Data:", batch_data)
    print("Labels:", batch_labels)

Data: tensor([[  353,   301,    41,  ...,     0,     0,     0],
        [  231,   436,    76,  ...,     0,     0,     0],
        [  937,   607,   778,  ...,     0,     0,     0],
        ...,
        [ 1837,  1667,    92,  ...,     0,     0,     0],
        [29159,  2864,   655,  ...,     0,     0,     0],
        [ 1310,     5,    98,  ...,     0,     0,     0]])
Labels: tensor([2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2,
        2, 2, 1, 1, 1, 1, 2, 2])
Data: tensor([[    4,    11,  2846,  ...,     0,     0,     0],
        [   23,   886, 10433,  ...,     0,     0,     0],
        [   43,   334,  1159,  ...,     0,     0,     0],
        ...,
        [  566,    96,   133,  ...,     0,     0,     0],
        [ 5426,    49,     0,  ...,     0,     0,     0],
        [   55,     4,    11,  ...,     0,     0,     0]])
Labels: tensor([1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2,
        1, 2, 1, 1, 1, 1, 1, 1])
Data: tensor([[12893, 

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

class SentimentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.0):
        super(SentimentRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)

        # Define the fully connected layer for sentiment prediction
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
      # Initialize hidden and cell states
      h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).to(self.lstm.weight_ih_l0.dtype)
      c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).to(self.lstm.weight_ih_l0.dtype)

      # Convert input tensor to the same data type as the LSTM weights
      x = x.to(self.lstm.weight_ih_l0.dtype)
      out, _ = self.lstm(x, (h0, c0))

      # Extract the output of the last time step
      out = out[:, -1, :]

      # Pass the output through the fully connected layer for sentiment prediction
      out = self.fc(out)

      return out

def train(model, train_dataloader, learning_rate, epoch_num):
    device = torch.device("cpu")
    model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    epochs = epoch_num
    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for batch in train_dataloader:
            # Move the batch to the device
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            # print(inputs)
            # print(labels)
        #     # Reshape the inputs to match the expected shape
            inputs = inputs.unsqueeze(1)  # Add an extra dimension for sequence length
            # print(inputs)
            # Clear the gradients
            optimizer.zero_grad()

        #     # Forward pass
            outputs = model(inputs)

        #     # Reshape the outputs and labels
            outputs = outputs.view(-1, outputs.size(-1))
            labels = labels.view(-1)

        #     # Convert labels to torch.long
            labels = labels.to(torch.long)
        #     # Calculate the loss
            loss = loss_fn(outputs, labels)

        #     # Backward pass and optimization
            loss.backward()
            optimizer.step()

        #     # Update the running loss
            running_loss += loss.item()

        #     # Calculate the number of correct predictions
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
            print("Predicted: ", predicted)
            print("Label: ",label)

        # # Calculate the training accuracy
        training_accuracy = correct_predictions / total_predictions

        # # Print the average loss and accuracy for the epoch
        epoch_loss = running_loss / len(train_dataloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss}, Accuracy: {training_accuracy * 100}%")

    return training_accuracy

In [28]:
first_model = SentimentRNN(198,64, 1, 3, 0.2)
train(first_model, train_data_loader, 0.0001, 80)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Streaming output truncated to the last 5000 lines.
        1, 1, 1, 1, 1, 1, 2, 1])
Label:  1
Predicted:  tensor([1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
Label:  1
Predicted:  tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 1, 1, 1, 1, 1, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2,
        1, 1, 1, 1, 1, 2, 1, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
    

0.6092695514092894

In [ ]:
len(vocab_dict)

23756

In [ ]:
pip install hyperopt

In [ ]:
from hyperopt import hp, fmin, tpe, Trials

# Define the hyperparameter search space
space = {
    'lr': hp.choice('lr', [1e-5, 1e-4, 1e-3, 1e-2, 5e-5, 5e-4]),
    'hidden_size': hp.choice('hidden_size', [32, 64, 128, 256]),
    'num_layers': hp.choice('num_layers', [1, 2, 3]),
    'dropout': hp.choice('dropout', [0.0, 0.2, 0.5, 0.8]),  # Dropout rates to choose from
    'epochs': hp.choice('epochs', [5, 10, 15, 20, 25]),
}

# Define the objective function to optimize (in this case, the negative accuracy)
def objective(params):
    lr = params['lr']
    hidden_size = params['hidden_size']
    num_layers = params['num_layers']
    dropout = params['dropout']
    epochs = params['epochs']

    # Create the model with the given hyperparameters
    model = SentimentRNN(198, hidden_size, num_layers, 3, dropout)

    # Train the model and get the accuracy
    accuracy = train(model, train_data_loader, lr, epochs)

    # Since hyperopt minimizes the objective function, we need to return the negative accuracy
    return -accuracy

# Initialize the Trials object to track the optimization process
trials = Trials()

# Run the hyperparameter search using the Tree of Parzen Estimators (TPE) algorithm
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# Print the best hyperparameters found
best_lr = [1e-5, 1e-4, 1e-3, 1e-2, 5e-5, 5e-4][best['lr']]
best_hidden_size = [32, 64, 128, 256][best['hidden_size']]
best_num_layers = [1, 2, 3][best['num_layers']]
best_dropout = [0.0, 0.2, 0.5, 0.8][best['dropout']]  # Map back to the specific dropout values
best_epochs = [5, 10, 15, 20, 25][best['epochs']]

print("Best Hyperparameters:")
print(f"Learning Rate: {best_lr}")
print(f"Hidden Layer Size: {best_hidden_size}")
print(f"Number of Hidden Layers: {best_num_layers}")
print(f"Dropout Rate: {best_dropout}")
print(f"Epochs: {best_epochs}")

Epoch [1/10], Loss: 1.0380532722368168, Accuracy: 46.22798887859128%
Epoch [1/25], Loss: 0.7928842247592303, Accuracy: 54.508142459949696%
Epoch [1/25], Loss: 1.0812074942269838, Accuracy: 45.48656163113994%
Epoch [1/25], Loss: 1.0570078197926611, Accuracy: 53.28743545611015%
  4%|▍         | 4/100 [00:24<08:29,  5.31s/trial, best loss: -0.5450814245994969]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "



Epoch [1/25], Loss: 1.165725460100941, Accuracy: 30.462068052429498%
Epoch [1/15], Loss: 0.8169071432597349, Accuracy: 52.47980934727923%
Epoch [1/10], Loss: 0.8164878892858184, Accuracy: 52.91936978683967%
Epoch [1/5], Loss: 1.0725192943434914, Accuracy: 47.051502714153315%
Epoch [1/5], Loss: 1.1175973984672294, Accuracy: 12.44273798490666%
Epoch [1/15], Loss: 0.9182759843873938, Accuracy: 53.44896067787634%
Epoch [1/25], Loss: 1.093213551131272, Accuracy: 39.38567456639746%
Epoch [1/5], Loss: 1.1226683945902114, Accuracy: 36.24520058254998%
Epoch [1/10], Loss: 1.0768269822316892, Accuracy: 53.292731365020515%
Epoch [1/15], Loss: 1.0584541292739662, Accuracy: 44.92519528664107%
Epoch [1/15], Loss: 0.8178099499830849, Accuracy: 53.2953793194757%
 15%|█▌        | 15/100 [01:33<08:13,  5.80s/trial, best loss: -0.5450814245994969]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "



Epoch [1/5], Loss: 0.731773883280968, Accuracy: 56.84628624387661%
Epoch [1/25], Loss: 1.124225925833971, Accuracy: 38.223222560571955%
Epoch [1/20], Loss: 1.127676176227421, Accuracy: 35.65735469349927%
Epoch [1/20], Loss: 1.061163991008342, Accuracy: 50.42234873560175%
Epoch [1/20], Loss: 1.052425626069181, Accuracy: 43.233152389778894%
Epoch [1/5], Loss: 0.8129472561956562, Accuracy: 53.30597113729644%
Epoch [1/5], Loss: 0.8013640792565665, Accuracy: 53.90440884416788%
Epoch [1/25], Loss: 1.0396832840408339, Accuracy: 45.9870250231696%
Epoch [1/5], Loss: 0.8028104902323174, Accuracy: 54.261882695617636%
Epoch [1/25], Loss: 0.9014950370405005, Accuracy: 53.27684363828942%
Epoch [1/5], Loss: 0.8191670504896243, Accuracy: 53.07824705415067%
Epoch [1/5], Loss: 0.7131311749119964, Accuracy: 57.30967827353369%
Epoch [1/5], Loss: 0.9851528214863253, Accuracy: 51.1929034820601%
Epoch [1/5], Loss: 0.9139330681353479, Accuracy: 54.304249966900564%
Epoch [1/10], Loss: 1.0544592588761417, Accur

In [33]:
best_model_1 = SentimentRNN(198,256, 1, 3, 0.8)
train(best_model_1, train_data_loader, 0.01, 120)

Streaming output truncated to the last 5000 lines.
        1, 1, 1, 1, 1, 1, 1, 1])
Label:  1
Predicted:  tensor([1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
        1, 2, 1, 1, 1, 1, 1, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2,
        1, 1, 2, 1, 1, 1, 2, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 2])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 2, 1, 1, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        2, 1, 1, 1, 1, 1, 2, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
Label:  1
Predicted:  tensor([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
    

0.6081579992060342

In [ ]:
best_model_1 = SentimentRNN(198,256, 1, 3, 0.8)
train(best_model_1, train_data_loader, 0.01, 15)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch [1/15], Loss: 0.7142480335732216, Accuracy: 57.20376009532636%
Epoch [2/15], Loss: 0.6789817371982119, Accuracy: 58.19674301602012%
Epoch [3/15], Loss: 0.6747321340111128, Accuracy: 58.80842049516748%
Epoch [4/15], Loss: 0.6728096632610228, Accuracy: 59.00172117039587%
Epoch [5/15], Loss: 0.6727192410970522, Accuracy: 58.87726731100224%
Epoch [6/15], Loss: 0.6726984823285069, Accuracy: 58.84284390308486%
Epoch [7/15], Loss: 0.6716506628037306, Accuracy: 58.82695617635376%
Epoch [8/15], Loss: 0.672617434799217, Accuracy: 58.70515027141533%
Epoch [9/15], Loss: 0.6713592113830799, Accuracy: 58.81106844962266%
Epoch [10/15], Loss: 0.6722241425594973, Accuracy: 58.78194095061565%
Epoch [11/15], Loss: 0.6711202736732619, Accuracy: 59.15000661988614%
Epoch [12/15], Loss: 0.6701084546473549, Accuracy: 59.0282007149477%
Epoch [13/15], Loss: 0.670084957295207, Accuracy: 59.00172117039587%
Epoch [14/15], Loss: 0.669825939078335, Accuracy: 59.00701707930624%
Epoch [15/15], Loss: 0.6699837934

0.5897524162584403

In [ ]:
test = SentimentRNN(198,128, 3, 3, 0.5)
train(test, train_data_loader, 0.00001, 15)

Epoch [1/15], Loss: 1.1078778646640308, Accuracy: 4.45385939361843%
Epoch [2/15], Loss: 1.1056316126008643, Accuracy: 11.065801668211307%
Epoch [3/15], Loss: 1.1033694718875287, Accuracy: 21.689394942406988%
Epoch [4/15], Loss: 1.1011361097704042, Accuracy: 34.230107242155434%
Epoch [5/15], Loss: 1.0989285245171854, Accuracy: 44.6180325698398%
Epoch [6/15], Loss: 1.096739778470226, Accuracy: 50.13901760889713%
Epoch [7/15], Loss: 1.0946216684191232, Accuracy: 52.60161525221766%
Epoch [8/15], Loss: 1.0924335764586977, Accuracy: 53.12855818879916%
Epoch [9/15], Loss: 1.090330648987501, Accuracy: 53.27949159274461%
Epoch [10/15], Loss: 1.0881850556939663, Accuracy: 53.2953793194757%
Epoch [11/15], Loss: 1.0861404205559675, Accuracy: 53.292731365020515%
Epoch [12/15], Loss: 1.0840671657001841, Accuracy: 53.29802727393089%
Epoch [13/15], Loss: 1.082031269481281, Accuracy: 53.319210909572355%
Epoch [14/15], Loss: 1.0799268695078696, Accuracy: 53.292731365020515%
Epoch [15/15], Loss: 1.077935

0.5330861909175162

In [ ]:
best_model_1 = SentimentRNN(198,128, 3, 3, 0.5)
train(best_model_1, train_data_loader, 0.00001, 15)

Epoch [1/15], Loss: 1.0771262763820393, Accuracy: 50.112538064345294%
Epoch [2/15], Loss: 1.0751675222002783, Accuracy: 50.189328743545616%
Epoch [3/15], Loss: 1.0732052108173549, Accuracy: 50.21051237918708%
Epoch [4/15], Loss: 1.0712776416242376, Accuracy: 50.634185092016416%
Epoch [5/15], Loss: 1.0693316136650066, Accuracy: 50.69508804448564%
Epoch [6/15], Loss: 1.0673842360466643, Accuracy: 51.22997484443268%
Epoch [7/15], Loss: 1.0654834912652993, Accuracy: 51.14259234741162%
Epoch [8/15], Loss: 1.0635840181169824, Accuracy: 51.43651529193698%
Epoch [9/15], Loss: 1.0616742283686251, Accuracy: 51.50006619886138%
Epoch [10/15], Loss: 1.0598265840076573, Accuracy: 51.50271415331657%
Epoch [11/15], Loss: 1.0579806388585271, Accuracy: 52.19912617502979%
Epoch [12/15], Loss: 1.056079328918134, Accuracy: 52.31828412551304%
Epoch [13/15], Loss: 1.0542890254367114, Accuracy: 52.14351913147094%
Epoch [14/15], Loss: 1.0525430643790663, Accuracy: 52.54600820865881%
Epoch [15/15], Loss: 1.0507

0.5261750297894876

In [ ]:
from hyperopt import hp, fmin, tpe, Trials

# Define the hyperparameter search space
space = {
    'lr': hp.choice('lr', [1e-5, 1e-4, 1e-3, 1e-2, 5e-5, 5e-4]),
    'hidden_size': hp.choice('hidden_size', [32, 64, 128, 256]),
    'num_layers': hp.choice('num_layers', [1, 2, 3]),
    'epochs': hp.choice('epochs', [5, 10, 15, 20, 25]),
}

# Define the objective function to optimize (in this case, the negative average accuracy)
def objective(params):
    lr = params['lr']
    hidden_size = params['hidden_size']
    num_layers = params['num_layers']
    epochs = params['epochs']

    # Create the model with the given hyperparameters
    model = SentimentRNN(198, hidden_size, num_layers, 3)

    # Train the model and get the average accuracy over multiple epochs
    num_epochs = 5  # You can change this value based on your computational resources
    accuracy_sum = 0.0
    for _ in range(num_epochs):
        accuracy = train(model, train_data_loader, lr, epochs)
        if accuracy is not None:  # Check if the accuracy is valid (not None)
            accuracy_sum += accuracy

    # Calculate the average accuracy
    avg_accuracy = accuracy_sum / num_epochs

    # Since hyperopt minimizes the objective function, we need to return the negative average accuracy
    return -avg_accuracy

# Initialize the Trials object to track the optimization process
trials = Trials()

# Run the hyperparameter search using the Tree of Parzen Estimators (TPE) algorithm
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# Print the best hyperparameters found
best_lr = [1e-5, 1e-4, 1e-3, 1e-2, 5e-5, 5e-4][best['lr']]
best_hidden_size = [32, 64, 128, 256][best['hidden_size']]
best_num_layers = [1, 2, 3][best['num_layers']]
best_epochs = [5, 10, 15, 20, 25][best['epochs']]

print("Best Hyperparameters:")
print(f"Learning Rate: {best_lr}")
print(f"Hidden Layer Size: {best_hidden_size}")
print(f"Number of Hidden Layers: {best_num_layers}")
print(f"Epochs: {best_epochs}")

Epoch [1/5], Loss: 1.1272812032699584, Accuracy: 34.62088130210401%
Epoch [2/5], Loss: 1.101588669315217, Accuracy: 39.346963080587535%
Epoch [3/5], Loss: 1.0797865278380259, Accuracy: 42.8483525208416%
Epoch [4/5], Loss: 1.0596483939034598, Accuracy: 45.60539896784439%
Epoch [5/5], Loss: 1.0397109879009307, Accuracy: 47.850337435490275%
Epoch [1/5], Loss: 1.0241256140905712, Accuracy: 49.46010321556173%
Epoch [2/5], Loss: 1.0086037973752098, Accuracy: 51.06391425168718%
Epoch [3/5], Loss: 0.9948258384068807, Accuracy: 52.0444620881302%
Epoch [4/5], Loss: 0.9817376621564229, Accuracy: 52.93370385073442%
Epoch [5/5], Loss: 0.9695022156881907, Accuracy: 53.66613735609369%
Epoch [1/5], Loss: 0.9578872815389482, Accuracy: 54.132592298531165%
Epoch [2/5], Loss: 0.9476227222170148, Accuracy: 54.80349344978166%
Epoch [3/5], Loss: 0.9383664737050496, Accuracy: 55.2123858674077%
Epoch [4/5], Loss: 0.9293087942638094, Accuracy: 55.60738388249305%
Epoch [5/5], Loss: 0.9204566982435801, Accuracy: 